In [63]:
import re

def is_function(sentence):
    """Check if the sentence contains a function and extract function name and parameters."""
    match = re.match(r'(\w+)\((.*)\)', sentence)
    if match:
        function_name = match.group(1)
        parameters = match.group(2).split(',')
        return function_name, [param.strip() for param in parameters]
    return None

def replace_with_theta(value, theta):
    """Recursively replace values in theta with their corresponding keys."""
    # Replace with substitutions from theta, both keys and values
    while value in theta:
        value = theta[value]
    return value

def unify_parameters(param1, param2, theta={}):
    """Unify two parameters. At least one must be a variable (uppercase or lowercase)."""
    param1 = replace_with_theta(param1, theta)
    param2 = replace_with_theta(param2, theta)

    if param1 == param2:
        return theta  # No unification needed
    elif is_variable(param1) and (not is_variable(param2) or is_function(param2)):
        return unify_var(param1, param2, theta)
    elif is_variable(param2) and (not is_variable(param1) or is_function(param1)):
        return unify_var(param2, param1, theta)
    elif is_function(param1) and is_function(param2):
        name1, params1 = is_function(param1)
        name2, params2 = is_function(param2)

        if name1 != name2 or len(params1) != len(params2):
            return "failure"

        # Unify each parameter
        for p1, p2 in zip(params1, params2):
            result = unify_parameters(p1, p2, theta)
            if result == "failure":
                return "failure"

        # Handle the parameter substitutions for functions
        for i in range(len(params1)):
            if is_function(params1[i]) and is_function(params2[i]):
                # Unify their parameters
                inner_theta = unify_parameters(params1[i], params2[i], theta)
                if inner_theta == "failure":
                    return "failure"
                theta.update(inner_theta)

                # Now replace parameters in theta
                for j in range(len(params1)):
                    # Replace parameters with corresponding values or keys
                    params1[j] = replace_with_theta(params1[j], theta)
                    params2[j] = replace_with_theta(params2[j], theta)

        return theta

    return "failure"

def unify_var(var, value, theta):
    """Unify a variable with a value. If already in the substitution map, check consistency."""
    if var in theta:
        return unify_parameters(theta[var], value, theta)
    elif value in theta:
        return unify_parameters(var, theta[value], theta)
    else:
        theta[var] = value
        print(f"{var}/{value}")  # Print the unification step
        return theta

def is_variable(x):
    """Check if the term is a variable (by convention, lowercase or uppercase letters represent variables)."""
    return type(x) == str and len(x) == 1 and (x.islower() or x.isupper())

def unify_sentences(sentence1, sentence2):
    """Unify two sentences if they are functions with the same name. Handles any number of parameters."""
    func1 = is_function(sentence1)
    func2 = is_function(sentence2)

    if func1 and func2:
        name1, params1 = func1
        name2, params2 = func2

        # Check if function names are the same
        if name1 != name2 or len(params1) != len(params2):
            return "failure"

        theta = {}
        for p1, p2 in zip(params1, params2):
            result = unify_parameters(p1, p2, theta)
            if result == "failure":
                return "failure"

        # Final cleanup of theta to replace all instances
        for key in list(theta.keys()):
            theta[key] = replace_with_theta(theta[key], theta)

        return theta
    return "failure"

# Testing the implementation
sentence1 = "Prime(13)"
sentence2 = "Prime(y)"

sentence3 = "Knows(John, x)"
sentence4 = "Knows(y, Mother(y))"

sentence5 = "pro(b(x), X, f(g(Z)))"
sentence6 = "pro(Z, f(Y), f(Y))"

sentence7 = "Quick(a, g(x, a), f(y))"
sentence8 = "Quick(a, g(f(b), a), x)"

print("Unifying Prime(13) and Prime(y):")
result1 = unify_sentences(sentence1, sentence2)  # Expected output: y/13
print(result1)

print("\nUnifying Knows(John, x) and Knows(y, Mother(y)):")
result2 = unify_sentences(sentence3, sentence4)  # Expected output: y/John, x/Mother(John)
print(result2)

print("\nUnifying pro(b(x), X, f(g(Z))) and pro(Z, f(Y), f(Y)):")
result3 = unify_sentences(sentence5, sentence6)  # Expected output: {'Z': 'b(x)', 'X': 'f(Y)', 'Y': 'g(b)'}
print(result3)

print("\nUnifying Quick(a, g(x, a), f(y)) and Quick(a, g(f(b), a), x):")
result4 = unify_sentences(sentence7, sentence8)  # Expected output: failure
print(result4)


Unifying Prime(13) and Prime(y):
y/13
{'y': '13'}

Unifying Knows(John, x) and Knows(y, Mother(y)):
y/John
x/Mother(y)
{'y': 'John', 'x': 'Mother(y)'}

Unifying pro(b(x), X, f(g(Z))) and pro(Z, f(Y), f(Y)):
Z/b(x)
X/f(Y)
Y/g(Z)
{'Z': 'b(x)', 'X': 'f(Y)', 'Y': 'g(Z)'}

Unifying Quick(a, g(x, a), f(y)) and Quick(a, g(f(b), a), x):
failure
